# Define Imports

In [1]:
import json
import os
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
import argparse

# Change dir in jupyter notebook

In [2]:
#dvc-cc-hide
if os.getcwd().endswith('source'):
    os.chdir('..')

# Define Argparser

In [3]:
print()
print('Tensorflow runs on the GPU: ', tf.config.list_physical_devices('GPU'))
print()
parser = argparse.ArgumentParser();

Tensorflow runs on the GPU:  True


In [4]:
# define the training
parser.add_argument('-lr', '--learning-rate', type=float, help='', default=0.0001)
parser.add_argument('-b','--batch-size', type=int, help='', default=64)
parser.add_argument('--num-of-epochs', type=int, help='', default=1000);

In [5]:
# define the model structure
parser.add_argument('--activation-function', type=str, help='', default='relu')
parser.add_argument('--kernel-width', type=int, help='', default=3)
parser.add_argument('--average-kernels', type=int, help='', default=32)
parser.add_argument('--num-of-conv-layers', type=int, help='', default=5)
parser.add_argument('--kernel-increasing-factor', type=float, help='', default=1.2)
parser.add_argument('--maxpool-after-n-layer', type=int, help='', default=0)
parser.add_argument('--dropout-factor-after-conv', type=float, help='', default=0.1)
parser.add_argument('--dropout-factor-after-maxp', type=float, help='', default=0.25);

In [6]:
"""dvc-cc-show
args = parser.parse_args()
""";

In [7]:
#dvc-cc-hide
args = parser.parse_args("--num-of-epochs 5 -b 16".split())

# Define the model

In [8]:
# define the model
padding = 'same'

In [9]:
kernel2d = (args.kernel_width, args.kernel_width)

In [10]:
model = Sequential()
for i in range(args.num_of_conv_layers):
    kernels = args.average_kernels * (args.kernel_increasing_factor ** (i-(args.num_of_conv_layers/2.)))
    kernels = int(kernels+0.5)
    
    if i == 0:
        input_shape = list([96,96,3])
        # use_cropping:
        input_shape[0] -= 10
        input_shape[1] -= 10 
        
        model.add(Conv2D(kernels, kernel2d, padding=padding,
                 input_shape=input_shape))
    else:
        model.add(Conv2D(kernels, kernel2d, padding=padding))
    model.add(Activation(args.activation_function))
    if args.maxpool_after_n_layer > 0 and (i+1) % args.maxpool_after_n_layer == 0:
        model.add(MaxPooling2D(pool_size=(2, 2)))
        if args.dropout_factor_after_maxp > 0:
            model.add(Dropout(args.dropout_factor_after_maxp))
    elif args.dropout_factor_after_conv > 0:
        model.add(Dropout(args.dropout_factor_after_conv))

model.add(GlobalAveragePooling2D())
if args.dropout_factor_after_maxp > 0:
    model.add(Dropout(args.dropout_factor_after_maxp))


model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

optimizer = tf.keras.optimizers.Adam(args.learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy',tf.keras.metrics.AUC()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 86, 86, 20)        560       
_________________________________________________________________
activation (Activation)      (None, 86, 86, 20)        0         
_________________________________________________________________
dropout (Dropout)            (None, 86, 86, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 86, 86, 24)        4344      
_________________________________________________________________
activation_1 (Activation)    (None, 86, 86, 24)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 86, 86, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 86, 29)        6

# Define Data Loader

In [11]:
def next_data_pcam(x_path,y_path,bz=args.batch_size):
    x = h5py.File(x_path,'r', libver='latest',swmr=True)['x']
    y = h5py.File(y_path,'r', libver='latest', swmr=True)['y']

    datalen = len(x)
    while True:
        indizies = None
        while indizies is None or len(indizies) == bz:
            indizies = list(np.unique(sorted(np.random.randint(datalen,size=bz))))
        
        x_data = np.array(x[indizies])
        
        # normalize_input
        x_data = x_data/256.0
        
        #use_cropping
        r = np.random.randint(10)
        r2 = np.random.randint(10)
        x_data = x_data[:,r:-10+r,r2:-10+r2]
        
        # flip_input
        if np.random.randint(2) == 1:
            x_data = x_data[:,::-1]
        if np.random.randint(2) == 1:
            x_data = x_data[:,:,::-1]
        
        yield x_data, np.array([[1,0],[0,1]])[y[indizies][:,0,0,0]]

# Train the model

In [12]:
tensorboard = tf.keras.callbacks.TensorBoard('tensorboard')
history = model.fit_generator(next_data_pcam(
                             'data/camelyonpatch_level_2_split_train_x.h5',
                             'data/camelyonpatch_level_2_split_train_y.h5'),
                        validation_steps=10,
                        steps_per_epoch=30,
                        epochs=args.num_of_epochs,
                        validation_data=next_data_pcam(
                             'data/camelyonpatch_level_2_split_valid_x.h5',
                             'data/camelyonpatch_level_2_split_valid_y.h5'),
                        workers=1,
                        verbose=2,
                        use_multiprocessing=False,
                        callbacks=[tensorboard])

model.save_weights('tf_model.h5')


W1108 19:31:48.377960 139897357526848 training_generator.py:413] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.


Epoch 1/5


W1108 19:31:50.047948 139897357526848 callbacks.py:244] Method (on_train_batch_end) is slow compared to the batch update (0.122980). Check your callbacks.
W1108 19:31:54.714812 139897357526848 training_generator.py:413] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.


100/100 - 8s - loss: 0.6949 - accuracy: 0.4867 - auc: 0.4899 - val_loss: 0.6965 - val_accuracy: 0.4500 - val_auc: 0.4500
Epoch 2/5


W1108 19:32:00.757679 139897357526848 training_generator.py:413] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.


...

100/100 - 6s - loss: 0.6644 - accuracy: 0.6207 - auc: 0.6596 - val_loss: 0.6439 - val_accuracy: 0.5667 - val_auc: 0.6747
Epoch 4/5


W1108 19:32:12.821516 139897357526848 training_generator.py:413] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.


100/100 - 6s - loss: 0.6435 - accuracy: 0.6313 - auc: 0.6834 - val_loss: 0.6714 - val_accuracy: 0.5667 - val_auc: 0.6233
Epoch 5/5


W1108 19:32:18.809404 139897357526848 training_generator.py:413] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.


100/100 - 6s - loss: 0.6449 - accuracy: 0.6300 - auc: 0.6773 - val_loss: 0.6220 - val_accuracy: 0.6833 - val_auc: 0.7253


# Save output files

In [13]:
if not os.path.exists('outputs'):
    os.mkdir('outputs')

In [14]:
with open('outputs/all-history.json','w') as f:
    json.dump(str(history.history),f)

params = {}
for p in history.history:
    if p.find('loss') >= 0:
        params[p] = np.min(history.history[p])
    else:
        params[p] = np.max(history.history[p])
        
with open('outputs/history-summary.json','w') as f:
    json.dump(str(params),f)